In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Online and Batch predictions using Vertex AI Feature Store

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/feature_store/sdk-feature-store.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/feature_store/sdk-feature-store.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
    </td>
    <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/feature_store/sdk-feature-store.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>

## Overview

This notebook introduces Vertex AI Feature Store, a managed cloud service for machine learning engineers and data scientists to store, serve, manage and share machine learning features at a large scale.

This notebook assumes that you understand basic Google Cloud concepts such as [Project](https://cloud.google.com/storage/docs/projects), [Storage](https://cloud.google.com/storage) and [Vertex AI](https://cloud.google.com/vertex-ai/docs). Some machine learning knowledge is also helpful but not required.

Learn more about [Vertex AI Feature Store](https://cloud.google.com/vertex-ai/docs/featurestore).

### Objective

In this notebook, you learn how to use `Vertex AI Feature Store` to import feature data, and to access the feature data for both online serving and offline tasks, such as training.

This tutorial uses the following Google Cloud ML services:

- `Vertex AI Feature Store`

The steps performed include:

- Create featurestore, entity type, and feature resources.
- Import feature data into `Vertex AI Feature Store` resource.
- Serve online prediction requests using the imported features.
- Access imported features in offline jobs, such as training jobs.
- Use streaming ingestion to ingest small amount of data.

### Dataset

This notebook uses a movie recommendation dataset as an example throughout all the sessions. The task is to train a model to predict if a user is going to watch a movie and serve this model online.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage
* Cloud BigQuery

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Installation

Install the following packages required to execute this notebook.

In [ ]:
! pip3 install --upgrade google-cloud-aiplatform -q

### Colab only: Uncomment the following cell to restart the kernel.

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

3. [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). {TODO: Update the APIs needed for your tutorial. Edit the API names, and update the link to append the API IDs, separating each one with a comma. For example, container.googleapis.com,cloudbuild.googleapis.com}

4. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

#### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### Region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### Authenticate your Google Cloud account

Depending on your Jupyter environment, you may have to manually authenticate. Follow the relevant instructions below.

**1. Vertex AI Workbench**
* Do nothing as you are already authenticated.

**2. Local JupyterLab instance, uncomment and run:**

In [ ]:
# ! gcloud auth login

**3. Colab, uncomment and run:**

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

**4. Service account or other**
* See how to grant Cloud Storage permissions to your service account at https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples.

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [ ]:
BUCKET_URI = "gs://your-bucket-name-unique"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

## Installation

Install the packages required to execute this notebook.

In [ ]:
from google.cloud import aiplatform
from google.cloud.aiplatform import Feature, Featurestore

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project.

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

## Terminology and concept

### Featurestore data model

Vertex AI Feature Store organizes data with the following 3 important hierarchical concepts:
```
Featurestore -> Entity type -> Feature
```
* **Featurestore**: The place to store your features
* **Entity type**: Under a featurestore, an entity type describes an object to be modeled, real one or virtual one.
* **Feature**: Under an entity type, a feature describes an attribute of the entity type

The movie prediction example lets you create a featurestore called `movie_prediction`. This store has 2 entity types. `users` and `movies`. The `users` entity type has the `age`, `gender`, and `liked_genres` features. The `movies` entity type has the `titles`, `genres`, and `average rating` features.


## Create featurestore and define schemas

In [ ]:
FEATURESTORE_ID = "movie_prediction_unique"
INPUT_CSV_FILE = "gs://cloud-samples-data-us-central1/vertex-ai/feature-store/datasets/movie_prediction.csv"
ONLINE_STORE_FIXED_NODE_COUNT = 1

### Create featurestore

The method to create a featurestore returns a
[long-running operation](https://google.aip.dev/151) (LRO). An LRO starts an asynchronous job. LROs are returned for other API
methods too, such as updating or deleting a featurestore. Running the code cell creates a featurestore and print the process log.

In [ ]:
fs = Featurestore.create(
    featurestore_id=FEATURESTORE_ID,
    online_store_fixed_node_count=ONLINE_STORE_FIXED_NODE_COUNT,
    project=PROJECT_ID,
    location=REGION,
    sync=True,
)

Use the following function call to retrieve a featurestore and check that it has been created.


In [ ]:
fs = Featurestore(
    featurestore_name=FEATURESTORE_ID,
    project=PROJECT_ID,
    location=REGION,
)
print(fs.gca_resource)

### Create entity Type

Entity types can be created within the `Featurestore` class. Below, create the `users` and `movies` entity types. A process log is printed out.

In [ ]:
# Create the `users` entity type
users_entity_type = fs.create_entity_type(
    entity_type_id="users",
    description="Users entity",
)

In [ ]:
# Create the `movies` entity type
movies_entity_type = fs.create_entity_type(
    entity_type_id="movies",
    description="Movies entity",
)

To retrieve an entity type or check that it has been created use the [get_entity_type](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/featurestore/featurestore.py#L106) or [list_entity_types](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/featurestore/featurestore.py#L278) methods on the Featurestore object.


In [ ]:
users_entity_type = fs.get_entity_type(entity_type_id="users")
movies_entity_type = fs.get_entity_type(entity_type_id="movies")
print(users_entity_type)
print(movies_entity_type)

In [ ]:
fs.list_entity_types()

### Create feature
You can create features within each entity type. Use the `create_feature` method to add features to the `users` and `movies` entity types.

In [ ]:
# To create one feature at a time, use:
users_feature_age = users_entity_type.create_feature(
    feature_id="age",
    value_type="INT64",
    description="User age",
)

users_feature_gender = users_entity_type.create_feature(
    feature_id="gender",
    value_type="STRING",
    description="User gender",
)

users_feature_liked_genres = users_entity_type.create_feature(
    feature_id="liked_genres",
    value_type="STRING_ARRAY",
    description="An array of genres this user liked",
)

Use the [`list_features`](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/featurestore/entity_type.py#L349) method to list all the features of a given entity type.

In [ ]:
users_entity_type.list_features()

In [ ]:
movies_feature_configs = {
    "title": {
        "value_type": "STRING",
        "description": "The title of the movie",
    },
    "genres": {
        "value_type": "STRING",
        "description": "The genre of the movie",
    },
    "average_rating": {
        "value_type": "DOUBLE",
        "description": "The average rating for the movie, range is [1.0-5.0]",
    },
}

In [ ]:
movie_features = movies_entity_type.batch_create_features(
    feature_configs=movies_feature_configs,
)

## Search created features

While the `list_features` method lets you view all features for the same entity type,
the [`search`](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/featurestore/feature.py#L352) method in the `Feature` class searches across all featurestores and entity types in a given location (such as `us-central1`) and returns a list of features. This lets you discover features created by someone else.

You can query based on feature properties including feature ID, entity type ID, and feature description. You can also limit results by filtering based on a specific featurestore, feature value type, and/or label. Some search examples are shown below.

**Example of using the `search` method**

Use the following code snippet to search for all features within a feature store:


In [ ]:
my_features = Feature.search(query="featurestore_id={}".format(FEATURESTORE_ID))
my_features

Now, narrow down the search to features that are of type `DOUBLE`.

In [ ]:
double_features = Feature.search(
    query="value_type=DOUBLE AND featurestore_id={}".format(FEATURESTORE_ID)
)
double_features[0].gca_resource

Or, limit the search results to features with specific keywords in their ID and type.

In [ ]:
title_features = Feature.search(
    query="feature_id:title AND value_type=STRING AND featurestore_id={}".format(
        FEATURESTORE_ID
    )
)
title_features[0].gca_resource

## Import feature values

You need to import feature values before you can use them for online or offline serving. In this step, you learn how to import feature values by ingesting the values from GCS (Google Cloud Storage). You can also import feature values from BigQuery or a pandas dataFrame.


### Source data format and layout

BigQuery table/Avro/CSV are supported as input data types. No matter what format you are using, each imported entity *must* have an ID. Each entity can *optionally* have a timestamp, specifying when the feature values are generated. This notebook uses Avro as an input, located at this public [bucket](https://console.cloud.google.com/storage/browser/cloud-samples-data-us-central1/vertex-ai/feature-store/datasets). The Avro schemas are as follows:

**For the `users` entity**:
```
schema = {
  "type": "record",
  "name": "User",
  "fields": [
      {
       "name":"user_id",
       "type":["null","string"]
      },
      {
       "name":"age",
       "type":["null","long"]
      },
      {
       "name":"gender",
       "type":["null","string"]
      },
      {
       "name":"liked_genres",
       "type":{"type":"array","items":"string"}
      },
      {
       "name":"update_time",
       "type":["null",{"type":"long","logicalType":"timestamp-micros"}]
      },
  ]
 }
```

**For the `movies` entity**:
```
schema = {
 "type": "record",
 "name": "Movie",
 "fields": [
     {
      "name":"movie_id",
      "type":["null","string"]
     },
     {
      "name":"average_rating",
      "type":["null","double"]
     },
     {
      "name":"title",
      "type":["null","string"]
     },
     {
      "name":"genres",
      "type":["null","string"]
     },
     {
      "name":"update_time",
      "type":["null",{"type":"long","logicalType":"timestamp-micros"}]
     },
 ]
}
```

### Import feature values for `users` entity type

When importing, specify the following in your request:

*   IDs of the features to import
*   Data source URI
*   Data source format: BigQuery Table/Avro/CSV


In [ ]:
USERS_FEATURES_IDS = [feature.name for feature in users_entity_type.list_features()]
USERS_FEATURE_TIME = "update_time"
USERS_ENTITY_ID_FIELD = "user_id"
USERS_GCS_SOURCE_URI = (
    "gs://cloud-samples-data-us-central1/vertex-ai/feature-store/datasets/users.avro"
)
GCS_SOURCE_TYPE = "avro"
WORKER_COUNT = 1
print(USERS_FEATURES_IDS)

In [ ]:
users_entity_type.ingest_from_gcs(
    feature_ids=USERS_FEATURES_IDS,
    feature_time=USERS_FEATURE_TIME,
    entity_id_field=USERS_ENTITY_ID_FIELD,
    gcs_source_uris=USERS_GCS_SOURCE_URI,
    gcs_source_type=GCS_SOURCE_TYPE,
    worker_count=WORKER_COUNT,
    sync=False,
)

### Import feature values for `movies` entity type

Similarly, import feature values for the `movies` entity type into the featurestore.


In [ ]:
MOVIES_FEATURES_IDS = [feature.name for feature in movies_entity_type.list_features()]
MOVIES_FEATURE_TIME = "update_time"
MOVIES_ENTITY_ID_FIELD = "movie_id"
MOVIES_GCS_SOURCE_URI = (
    "gs://cloud-samples-data-us-central1/vertex-ai/feature-store/datasets/movies.avro"
)
GCS_SOURCE_TYPE = "avro"
WORKER_COUNT = 1
print(MOVIES_FEATURES_IDS)

In [ ]:
movies_entity_type.ingest_from_gcs(
    feature_ids=MOVIES_FEATURES_IDS,
    feature_time=MOVIES_FEATURE_TIME,
    entity_id_field=MOVIES_ENTITY_ID_FIELD,
    gcs_source_uris=MOVIES_GCS_SOURCE_URI,
    gcs_source_type=GCS_SOURCE_TYPE,
    worker_count=WORKER_COUNT,
    sync=False,
)

## Get online predictions from your model

[Online serving](https://cloud.google.com/vertex-ai/docs/featurestore/serving-online)
lets you serve feature values for small batches of entities. It's designed for latency-sensitive services, such as online model prediction. For example, for a movie service, you might want to quickly show movies that the current user would most likely watch.

### Read one entity per request

With the Python SDK, it's easy to read feature values of one entity. By default, the SDK returns the latest value of each feature, that is, the feature values with the most recent timestamps.

To read feature values, specify the entity type ID and features to read. By default all the features of an entity type are selected. The output response displays the selected entity type ID and the selected feature values as a Pandas dataframe.

In [ ]:
users_entity_type.read(entity_ids="bob")

In [ ]:
movies_entity_type.read(entity_ids="movie_01", feature_ids="title")

### Read multiple entities per request

To read feature values from multiple entities, specify the different entity type IDs. By default, all the features of an entity type are selected. Note that fetching only a small number of entities is recommended when using this SDK due to its latency-sensitive nature.

In [ ]:
users_entity_type.read(entity_ids=["bob", "alice"])

In [ ]:
movies_entity_type.read(
    entity_ids=["movie_02", "movie_03", "movie_04"], feature_ids=["title, genres"]
)

Now that you have learned how to fetch imported feature values for online serving, the next step is learning how to use imported feature values for offline use cases.

## Get batch predictions from your model

Batch serving is used to fetch a large batch of feature values for high-throughput, and is typically used for training a model or batch prediction. In this section, you learn how to prepare for training examples by using the Featurestore's batch serve function.

### Use case

**The task** is to prepare a training dataset to train a model, which predicts if a given user is going to watch a movie. To achieve this, you need 2 sets of input:

*   Features: you already imported into the featurestore.
*   Labels: the ground-truth data recorded that user X has watched movie Y.


To be more specific, the ground-truth observation is described in Table 1 and the desired training dataset is described in Table 2. Each row in Table 2 is a result of joining the imported feature values from Vertex AI Feature Store according to the entity IDs and timestamps in Table 1. In this example,  the `age`, `gender` and `liked_genres` features from `users` and
the `titles`, `genres` and `average_rating` features from `movies` are chosen to train the model. Note that only positive examples are shown in these 2 tables, that is, you can imagine there is a label column whose values are all `True`.

[`batch_serve_to_bq`](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/featurestore/featurestore.py#L770) takes Table 1 as
input, joins all required feature values from the featurestore, and returns Table 2 for training.

<h4 align="center">Table 1. Ground-truth data</h4>

users | movies | timestamp
----- | -------- | --------------------
alice  | Cinema Paradiso     | 2019-11-01T00:00:00Z
bob  | The Shining     | 2019-11-15T18:09:43Z
...   | ...      | ...


<h4 align="center">Table 2. Expected training data generated by using batch serve</h4>

timestamp            | entity_type_users | age | gender | liked_genres | entity_type_movies | title | genre | average_rating
-------------------- | ----------------- | --------------- | ---------------- | -------------------- | - | -------- | --------- | -----
2019-11-01T00:00:00Z | bob              | 35        | M                | [Action, Crime]      | movie_02            | The Shining | Horror | 4.8
2019-11-01T00:00:00Z | alice             | 55        | F                | [Drama, Comedy]      | movie_03          | Cinema Paradiso | Romance | 4.5 |
... | ... | ... | ... | ... | ... | ... | ... | ...


#### Why timestamp?

Note that there is a `timestamp` column in Table 2 to indicate the time when the ground-truth was observed. This is to avoid data inconsistency.

For example, the 2nd row of Table 2 indicates that user `alice` watched movie `Cinema Paradiso` on `2019-11-01T00:00:00Z`. The featurestore keeps feature values for all timestamps but fetches feature values *only* at the given timestamp during batch serving. On that day, Alice might have been 54 years old, but now Alice might be 56; featurestore returns `age=54` as Alice's age, instead of `age=56`, because that is the value of the feature at the observation time. Similarly, other features might be time-variant as well, such as `liked_genres`.

### Create BigQuery dataset for output

You need a BigQuery dataset to host the output data in `us-central1`. Input the name of the dataset you want to create and specify the name of the table you want to store the output created later. These are used in the next section.

**Make sure that the table name does NOT already exist**.


In [ ]:
from datetime import datetime

from google.cloud import bigquery

In [ ]:
# Output dataset
DESTINATION_DATA_SET = "movie_predictions"  # @param {type:"string"}
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
DESTINATION_DATA_SET = "{prefix}_{timestamp}".format(
    prefix=DESTINATION_DATA_SET, timestamp=TIMESTAMP
)

# Output table. Make sure that the table does NOT already exist; the BatchReadFeatureValues API cannot overwrite an existing table
DESTINATION_TABLE_NAME = "training_data"  # @param {type:"string"}

DESTINATION_PATTERN = "bq://{project}.{dataset}.{table}"
DESTINATION_TABLE_URI = DESTINATION_PATTERN.format(
    project=PROJECT_ID, dataset=DESTINATION_DATA_SET, table=DESTINATION_TABLE_NAME
)

In [ ]:
# Create dataset
client = bigquery.Client(project=PROJECT_ID)
dataset_id = "{}.{}".format(client.project, DESTINATION_DATA_SET)
dataset = bigquery.Dataset(dataset_id)
dataset.location = REGION
dataset = client.create_dataset(dataset)
print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

### Batch read feature values

Assemble the request which specifies the following info:

*   Where is the label data, i.e., Table 1.
*   Which features are read, i.e., the column names in Table 2.

The output is stored in the BigQuery table.


In [ ]:
SERVING_FEATURE_IDS = {
    # to choose all the features use 'entity_type_id: ['*']'
    "users": ["age", "gender", "liked_genres"],
    "movies": ["title", "average_rating", "genres"],
}

In [ ]:
fs.batch_serve_to_bq(
    bq_destination_output_uri=DESTINATION_TABLE_URI,
    serving_feature_ids=SERVING_FEATURE_IDS,
    read_instances_uri=INPUT_CSV_FILE,
)

After the LRO finishes, you should be able to see the result in the [BigQuery console](https://console.cloud.google.com/bigquery), as a new table under the BigQuery dataset created earlier.

## Streaming ingestion

Streaming ingestion is currently public preview.

Streaming ingestion lets you make real-time updates to feature values. While batch import is suitable for importing a large volume of data with high latency, streaming ingestion is suitable for ingesting small amount of data with low latency. The written data becomes available to read using batch export and online serving.

In [ ]:
# Since streaming ingestion is public preview, the feature is available in aiplatform_v1beta1.
from google.cloud.aiplatform_v1beta1 import (
    FeaturestoreOnlineServingServiceClient, FeaturestoreServiceClient)
from google.cloud.aiplatform_v1beta1.types import \
    featurestore_online_service as featurestore_online_service_pb2
from google.cloud.aiplatform_v1beta1.types import types as types_pb2

API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)
# Create client connection
admin_client = FeaturestoreServiceClient(client_options={"api_endpoint": API_ENDPOINT})
data_client = FeaturestoreOnlineServingServiceClient(
    client_options={"api_endpoint": API_ENDPOINT}
)

In [ ]:
# Call `write_feature_values` to ingest data to `users` entity type.
data_client.write_feature_values(
    entity_type=admin_client.entity_type_path(
        PROJECT_ID, REGION, FEATURESTORE_ID, "users"
    ),
    payloads=[
        featurestore_online_service_pb2.WriteFeatureValuesPayload(
            entity_id="1305",
            feature_values={
                "age": featurestore_online_service_pb2.FeatureValue(int64_value=34),
                "gender": featurestore_online_service_pb2.FeatureValue(
                    string_value="female"
                ),
                "liked_genres": featurestore_online_service_pb2.FeatureValue(
                    string_array_value=types_pb2.StringArray(values=["drama", "action"])
                ),
            },
        ),
        featurestore_online_service_pb2.WriteFeatureValuesPayload(
            entity_id="1306",
            feature_values={
                "age": featurestore_online_service_pb2.FeatureValue(int64_value=50),
                "gender": featurestore_online_service_pb2.FeatureValue(
                    string_value="male"
                ),
                "liked_genres": featurestore_online_service_pb2.FeatureValue(
                    string_array_value=types_pb2.StringArray(
                        values=["suspense", "comedy"]
                    )
                ),
            },
        ),
    ],
)

Upon successful completion, the `write_feature_values` API returns an empty response.
Similarly, ingest data to the `movies` entity type

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

You can also keep the project, but delete the featurestore and the BigQuery dataset by running the following code:

In [ ]:
# Delete Featurestore
fs.delete(force=True)

In [ ]:
# Delete BigQuery dataset
client = bigquery.Client(project=PROJECT_ID)
client.delete_dataset(
    DESTINATION_DATA_SET, delete_contents=True, not_found_ok=True
)  # Make an API request.

print("Deleted dataset '{}'.".format(DESTINATION_DATA_SET))